### Importação de bibliotecas

In [9]:
# Importação das bibliotecas necessárias para o projeto
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import folium
import random

### Classe LondonNetworkGraph

In [10]:
# Criação da classe para analisar a rede do metro de Londres
class LondonNetworkGraph:

    def __init__(self):
        # Inicialização do grafo direcionado do networkx (escolha do MultiDiGraph por ser um grafo direcional capaz de lidar com arestas paralelas)
        self.graph = nx.MultiDiGraph()
        # Dicionário que irá guardar a informação relativas às conexões e os seus pesos
        self.weights = {}

    # Método para obter o nome da estação a partir do seu ID
    def get_station_name(self, station_id):
        return self.graph.nodes[station_id]['name']


    def stations(self, file_path):
        # Leitura do ficheiro com as estações usando a biblioteca pandas
        stations_dataframe = pd.read_csv(file_path)

        # Guardar as informações de cada estação em diferentes variáveis
        for index, row in stations_dataframe.iterrows():
            station_id = row['id']
            latitude = row['latitude']
            longitude = row['longitude']
            name = row['name']
            display_name = row['display_name']
            zone = row['zone']
            total_lines = row['total_lines']
            rail = row['rail']

            # Adicionar a estação ao grafo
            self.graph.add_node(station_id, latitude=latitude, longitude=longitude, name=name, display_name=display_name,
                    zone=zone, total_lines=total_lines, rail=rail)


    def connections(self, file_path):
        # Leitura do ficheiro das conexões usando a biblioteca pandas
        connections_dataframe = pd.read_csv(file_path)

        # Guardar as informações das ligações em diferentes variáveis
        for index, row in connections_dataframe.iterrows():
            line = row['Line']
            from_station = row['From Station Id']
            to_station = row['To Station Id']
            distance = row['Distance (Kms)']
            off_peak_time = row['Off Peak Running Time (mins)']
            am_peak_time = row['AM peak (0700-1000) Running Time (Mins)']
            inter_peak_time = row['Inter peak (1000 - 1600) Running time (mins)']

            # Adicionar a ligação ao grafo
            self.graph.add_edge(from_station, to_station, line=line, distance=distance, off_peak_time=off_peak_time,
                    am_peak_time=am_peak_time, inter_peak_time=inter_peak_time)
            
            # Armazenar o peso da conexão no dicionário self.weights (é necessário incluir a line na chave já que pode ir-se de uma estação de partida para uma de chegada em linhas diferentes)
            self.weights[(from_station, to_station, line)] = (distance, off_peak_time, am_peak_time, inter_peak_time)


    def n_stations(self):
        # Devolver o número total de estações
        return self.graph.number_of_nodes()
    

    def n_stations_zone(self):
            # Guardar o número de estações por zona criando um dicionário
            stations_by_zone = {}

            # Percorrer todas as estações do grafo e contar quantas estações existem em cada zona guardando-as no dicionário
            for station_id in self.graph.nodes():
                # Extrair a parte inteira da zona
                zone = self.graph.nodes[station_id]['zone']
                lower_zone = int(zone)
                # Obter o inteiro imediatamente a seguir
                upper_zone = lower_zone + 1

                # Caso a zona já exista no dicionário, adiciona mais um ao contador da estação
                if lower_zone in stations_by_zone:
                    stations_by_zone[lower_zone] += 1
                # Caso não exista, cria no dicionário a zona e inicializa com o valor 1
                else:
                    stations_by_zone[lower_zone] = 1

                # Verifica se a zona possui parte fracionária 
                if zone % 1 != 0: 
                    # Se tiver e a upper_zone já existir acrescenta mais um ao seu contador
                    if upper_zone in stations_by_zone:
                        stations_by_zone[upper_zone] += 1
                    # Se tiver mas a upper_zone não existir no dict, cria a estação com o valor 1
                    else:
                        stations_by_zone[upper_zone] = 1
                        
            # Devolve um dicionário onde as chaves são as zonas e os valores são o número de estações em cada zona
            return stations_by_zone
    

    def n_edges(self):
        # Devolver o número total de arestas
        return self.graph.number_of_edges()
    

    def n_edges_line(self):
        # Dicionário para guardar o número de arestas por linha
        edges_by_line = {}

        # Percorrer todas as arestas do grafo e contar quantas existem por linha guardando-as no dicionário
        for node_A, node_B, key in self.graph.edges(keys=True):
            line = self.graph[node_A][node_B][key]['line']
            # Se a linha já estiver presente no dicionário, incrementa o contador de arestas para essa linha
            if line in edges_by_line:
                edges_by_line[line] += 1
            # Caso contrário, adiciona a linha ao dicionário e inicializa o contador para 1
            else:
                edges_by_line[line] = 1

        # devolver um dicionário onde as chaves são as linhas e os valores são o número de edges
        return edges_by_line


    def mean_degree(self):
        # Calcula a média dos graus dividindo o total de arestas pelo número de estações
        return self.n_edges() / self.n_stations()
    
    
    def mean_weight(self, weight):
        # Variável para acumular a soma de todos os elementos de determinado peso 
        total_weight = 0
        # Variável para contar o número de elementos de um determinado peso
        count = 0

        # Percorrer e obter todos os dados guardados das conexões 
        for node_A, node_B, data in self.graph.edges(data=True):
            # Verificar se o peso passado no parâmetro existe nos dados das conexões
            if weight in data:
                # Soma de todos os elementos do peso 
                total_weight += data[weight]
                # Contagem de todos os elementos do peso
                count += 1
        
        # Devolver a média do peso em questão
        if count > 0:
            return total_weight / count
        else:
            return 0
        

    def visualize(self, visualization='folium'):
        # Ajustar o tamanho da figura
        plt.figure(figsize=(14, 10))

        # Ajustar o tamanho dos nós
        node_size = 250

        # Visualizar o grafo usando Networkx
        if visualization == 'networkx':
            nx.draw(self.graph, with_labels=True, node_size=node_size)
            plt.show()
        # Visualizar o grafo usando Matplotlib
        elif visualization == 'matplotlib':
            # Obter as posições de cada vértice
            node_positions = {node_id: (self.graph.nodes[node_id]['longitude'], self.graph.nodes[node_id]['latitude'])
                            for node_id in self.graph.nodes}

            # Criar a nova figura com a posição ajustada
            fig, ax = plt.subplots(figsize=(14, 10))

            # Desenhar as arestas
            for edge in self.graph.edges:
                node_a = edge[0]
                node_b = edge[1]
                x = [node_positions[node_a][0], node_positions[node_b][0]]
                y = [node_positions[node_a][1], node_positions[node_b][1]]
                ax.plot(x, y, color='gray', linewidth=0.5, alpha=0.5)

            # Desenhar os vértices
            for node_id, pos in node_positions.items():
                ax.plot(pos[0], pos[1], 'o', color='darkblue', markersize=5)

            # Adicionar labels aos vértices
            for node_id, pos in node_positions.items():
                ax.text(pos[0], pos[1], self.graph.nodes[node_id]['name'], fontsize=10)

            # Definir os eixos das labels e do título
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            ax.set_title('London Metro Network Graph')

            # Remover os ticks dos eixos
            ax.set_xticks([])
            ax.set_yticks([])

            # Ajustar os limites para o plot
            ax.set_xlim(min(node_positions.values(), key=lambda x: x[0])[0] - 0.01,
                        max(node_positions.values(), key=lambda x: x[0])[0] + 0.01)
            ax.set_ylim(min(node_positions.values(), key=lambda x: x[1])[1] - 0.01,
                        max(node_positions.values(), key=lambda x: x[1])[1] + 0.01)

            # Apresentar o grafo
            plt.tight_layout()
            plt.show()
        # Visualizar o grafo usando Folium
        elif visualization == 'folium':
            # Calcular o centro do mapa
            center_lat = sum(node['latitude'] for node in self.graph.nodes.values()) / len(self.graph.nodes)
            center_lon = sum(node['longitude'] for node in self.graph.nodes.values()) / len(self.graph.nodes)

            # Criar um objeto de mapa Folium
            map_graph = folium.Map(location=[center_lat, center_lon], zoom_start=12)

            # Adicionar as arestas ao mapa
            for edge in self.graph.edges:
                node_a = edge[0]
                node_b = edge[1]
                coords = [(self.graph.nodes[node_a]['latitude'], self.graph.nodes[node_a]['longitude']),
                        (self.graph.nodes[node_b]['latitude'], self.graph.nodes[node_b]['longitude'])]

                # Criar o pop-up com as informações da aresta
                popup_text = f"Line: {self.graph.edges[edge]['line']}"

                # Adicionar a aresta com o pop-up ao mapa
                folium.PolyLine(coords, color='darkblue', weight=2, opacity=0.5, popup=popup_text).add_to(map_graph)

            # Adicionar os nós ao mapa
            for node_id in self.graph.nodes:
                node = self.graph.nodes[node_id]
                folium.Marker(location=[node['latitude'], node['longitude']],
                                    popup=node['name']).add_to(map_graph)

            # Exibir o mapa
            display(map_graph)


    # Algoritmo do Dijkstra (implementação em Python)
    def shortest_path(self, source, target, peso):
        # Obter todas as estações
        start_nodes = [i[0] for i in self.weights]
        end_nodes = [i[1] for i in self.weights]
        all_nodes = set(start_nodes + end_nodes)
        # Dicionário para guardar as distâncias no cálculo do caminho mais curto 
        distances = {}
        # Dicionário para guardar os predecessores no cálculo do caminho mais curto 
        predecessors = {}
        # Inicializar distâncias como infinito para todas as estações, exceto a origem
        for station in all_nodes:
            distances[station] = float('inf')
        distances[source] = 0

        # Criar um conjunto para armazenar estações visitadas
        visited = set()

        # Iterar até que todas as estações tenham sido visitadas
        while len(visited) < len(all_nodes):
            # Encontrar a estação não visitada com a menor distância atual
            min_distance = float('inf')
            min_station = None
            for station in all_nodes:
                if station not in visited and distances[station] < min_distance:
                    min_distance = distances[station]
                    min_station = station

            # Adicionar a estação encontrada ao conjunto de visitadas
            visited.add(min_station)

            # Atualizar as distâncias e predecessores das estações vizinhas
            for neighbor in all_nodes:
                if neighbor not in visited:
                    for key in self.weights.keys():
                        if key[:2] == (min_station, neighbor):
                            edge_weight = self.weights[key][peso]
                            distance = distances[min_station] + edge_weight

                            if distance < distances[neighbor]:
                                distances[neighbor] = distance
                                predecessors[neighbor] = min_station

        # Verificar se existe um caminho válido entre a origem e o destino
            if source not in start_nodes:
                raise ValueError("Não existe um caminho válido entre a origem e o destino. A estação em questão não existe na lista de estações de origem.")
            elif target not in end_nodes:
                raise ValueError("Não existe um caminho válido entre a origem e o destino. A estação em questão não existe na lista de destinos.")
  
        # Reconstruir o caminho ótimo a partir do destino até a origem
        path = []
        current_station = target
        while current_station != source:
            path.append(current_station)
            current_station = predecessors[current_station]
        path.append(source)

        # Inverter o caminho para obter a ordem correta da origem ao destino
        path.reverse()
        return path
    
    # Cálculo da estação mais próxima
    def get_nearest_station(self, start, end):
        # Inicializa as menores distâncias para o início e para o fim como infinito
        min_start_distance = float('inf')
        min_end_distance = float('inf')
        # Inicializa as respetivas estações mais próximas como None
        nearest_start_station = None
        nearest_end_station = None

        # Percorre o dicionário de pesos
        for connection, weights in self.weights.items():
            # Guarda a informação da estação de partida e de chegada
            from_station, to_station,line = connection
            if from_station == start:
                # Utiliza o peso no indíce 0 dos valores do dicionário dos pesos, que corresponde à distância em kms
                start_distance = weights[0]  
                if start_distance < min_start_distance:
                    min_start_distance = start_distance
                    # Obtém o nome da estação através do método get_station_name que devolve o nome da estação com base no seu ID 
                    nearest_start_station = self.get_station_name(to_station)
            if from_station == end:
                # Utiliza o peso no indíce 0 dos valores do dicionário dos pesos, que corresponde à distância em kms
                end_distance = weights[0] 
                if end_distance < min_end_distance:
                    min_end_distance = end_distance
                    # Obtém o nome da estação através do método get_station_name que devolve o nome da estação com base no seu ID 
                    nearest_end_station = self.get_station_name(to_station)

        # Devolve a estação mais próxima da de partida e da de chegada, respetivamente
        return nearest_start_station, nearest_end_station


    # Visualizar o grafo com o caminho mais curto utilizando o Folium
    def visualize_graph_shortest_path(self, start, end, shortest_path):
        # Calcular o centro do mapa
        center_lat = sum(node['latitude'] for node in self.graph.nodes.values()) / len(self.graph.nodes)
        center_lon = sum(node['longitude'] for node in self.graph.nodes.values()) / len(self.graph.nodes)
        
        # Criar um objeto de mapa Folium
        map_graph = folium.Map(location=[center_lat, center_lon], zoom_start=12)

        # Adicionar as arestas ao mapa
        for edge in self.graph.edges():
            start_node = edge[0]
            end_node = edge[1]
            start_coords = [self.graph.nodes[start_node]['latitude'], self.graph.nodes[start_node]['longitude']]
            end_coords = [self.graph.nodes[end_node]['latitude'], self.graph.nodes[end_node]['longitude']]
            folium.PolyLine([start_coords, end_coords], color='grey', weight=2, opacity=0.5).add_to(map_graph)

        # Adicionar os vértices ao mapa
        for node in self.graph.nodes():
            station_name = self.get_station_name(node)
            folium.Marker([self.graph.nodes[node]['latitude'], self.graph.nodes[node]['longitude']],
                        popup=station_name).add_to(map_graph)

        # Adicionar marcadores para o ponto de início e de fim do caminho
        folium.Marker([self.graph.nodes[start]['latitude'], self.graph.nodes[start]['longitude']],
                    popup='Start Point', icon=folium.Icon(color='green')).add_to(map_graph)
        folium.Marker([self.graph.nodes[end]['latitude'], self.graph.nodes[end]['longitude']],
                    popup='End Point', icon=folium.Icon(color='red')).add_to(map_graph)

        # Adicionar ao mapa o caminho mais curto
        path_coords = [(self.graph.nodes[node]['latitude'], self.graph.nodes[node]['longitude']) for node in shortest_path]
        folium.PolyLine(path_coords, color='black', weight=4).add_to(map_graph)

        # Exibir o mapa
        display(map_graph)
    

    # Simulação final
    def generate_random_path(self, metodo='python'):
        # Obter as estações de partida
        from_stations = [i[0] for i in self.graph.edges()]
        # Obter as estações de chegada
        to_stations = [i[1] for i in self.graph.edges()]

        # Range para as coordenadas de x
        x1, x2 = min(from_stations), max(from_stations) 
        # Range para as coordenadas de y
        y1, y2 = min(to_stations), max(to_stations)  

        # Gerar os dois pontos aleatórios (start, end) com base nos ranges definidos anteriormente
        start = random.randint(x1, x2)
        end = random.randint(y1, y2)

        # Assegurar que o ponto de partida e destino para o caminho são diferentes
        while start == end:
            end = random.randint(y1, y2)

        # Gerar uma hora aleatória (entre 1 and 24)
        hour = random.randint(1, 24)

        # Imprimir pontos e hora aleatórios
        print("Random Hour:", hour)
        print("Start:", start)
        print("End:", end)

        # Calcular as estações mais próximas do início e do fim
        nearest_start_station, nearest_end_station = self.get_nearest_station(start, end)

        # Escolher o método para o algoritmo do Dijsktra (implementação em python ou através da biblioteca do networkx)
        if metodo == 'python':
            # Ajustar o peso tendo em conta o pico da manhã
            if hour >= 7 and hour <= 9:
                shortest_path = self.shortest_path(float(start), float(end), 2)
            # Ajustar o peso tendo em conta o tempo entre picos
            elif hour >= 10 and hour <= 16:
                shortest_path = self.shortest_path(float(start), float(end), 3)
            # Ajustar o peso tendo em conta o off_peak 
            else:
                shortest_path = self.shortest_path(float(start), float(end), 1)

        # Melhor caminho usando o algoritmo de Dijkstra do networkx
        elif metodo == 'networkx':
            # Ajustar o peso tendo em conta o pico da manhã
            if hour >= 7 and hour <= 9:
                weight_attribute = 'am_peak_time'
            # Ajustar o peso tendo em conta o tempo entre picos
            elif hour >= 10 and hour <= 16:
                weight_attribute = 'inter_peak_time'
            # Ajustar o peso tendo em conta o off_peak 
            else:
                weight_attribute = 'off_peak_time'
            
            # Melhor caminho usando o algoritmo de Dijkstra do NetworkX
            shortest_path = nx.shortest_path(self.graph, source=start, target=end, weight=weight_attribute)

        
        # Imprimir os resultados
        print("Shortest Path:", shortest_path)
        print("Nearest start station:", nearest_start_station)
        print("Nearest end station:", nearest_end_station)


        # Visualizar o grafo com o caminho mais curto
        self.visualize_graph_shortest_path(start, end, shortest_path)

### Testes dos métodos da classe

In [11]:
# TESTES 

# Criar o grafo
london_graph = LondonNetworkGraph()

# Adicionar as estações e as ligações aos vértices e arestas respetivamente
london_graph.stations('stations.csv')
london_graph.connections('connections.csv')

print("Nome da estação 1:", london_graph.get_station_name(1))

print("Número total de estações:", london_graph.n_stations())
print("Número de estações por zona:", london_graph.n_stations_zone())

print("Número total de ligações:", london_graph.n_edges())
print("Número de ligações por linha:", london_graph.n_edges_line())

print("Grau médio das estações:", london_graph.mean_degree())

# Para visualizar outros pesos ajustar o parâmetro para: 'distance', 'off_peak_time', 'am_peak_time' ou 'inter_peak_time'
print("Peso médio:", london_graph.mean_weight('off_peak_time'))

# Parâmetros para o melhor caminho: ponto de partida (float) + destino (float) + peso a considerar (distância=0, off_peak_time=1, 'am_peak_time'=2, 'inter_peak_time'=3)
print("Melhor caminho entre os pontos 252 e 74:", london_graph.shortest_path(252, 74, 0))

print("Estações mais próximas da partida 252 e da chegada 74:", london_graph.get_nearest_station(252, 74))

Nome da estação 1: Acton Town
Número total de estações: 302
Número de estações por zona: {3: 70, 1: 64, 2: 96, 4: 44, 5: 29, 6: 20, 7: 3, 10: 2, 9: 1, 8: 2}
Número total de ligações: 743
Número de ligações por linha: {10.0: 99, 4.0: 118, 8.0: 70, 3.0: 54, 6.0: 54, 9.0: 102, 1.0: 48, 7.0: 52, 2.0: 98, 12.0: 2, 11.0: 30, 5.0: 16}
Grau médio das estações: 2.4602649006622515
Peso médio: 2.0794885598923294
Melhor caminho entre os pontos 252 e 74: [252, 5.0, 194.0, 182.0, 73.0, 1.0, 110.0, 17.0, 293.0, 74]
Estações mais próximas da partida 252 e da chegada 74: ('Alperton', 'West Brompton')


### Visualização da rede do metro de Londres

In [12]:
# TESTES 

london_graph.visualize()

<Figure size 1008x720 with 0 Axes>

### Simulação final do caminho mais curto/rápido

In [13]:
# TESTES

london_graph.visualize_graph_shortest_path(252, 74, london_graph.shortest_path(252, 74, 0))

In [28]:
# TESTES PARA A SIMULAÇÃO FINAL
london_graph.generate_random_path('python')

Random Hour: 2
Start: 278
End: 281
Shortest Path: [278.0, 159.0, 143.0, 206.0, 137.0, 298.0, 113.0, 246.0, 281.0]
Nearest start station: Maida Vale
Nearest end station: North Wembley


Realizado por:
- João Dias nº 110305
- David Franco nº 110733 